In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import Dataset, random_split
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


if torch.cuda.is_available():

    device = torch.device("cuda")
    print("You are using GPU")
else:
    print("You are using CPU")

You are using GPU


In [4]:
df = pd.read_csv('data_processing/csv_file/landmark_7_9_2.csv')

# Define your PyTorch dataset class
class ProcessData(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.iloc[:, 1:].values
        self.labels = dataframe.iloc[:, 0].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = {
            'data': torch.tensor(self.data[idx], dtype=torch.float32, device=device),
            'label': torch.tensor(self.labels[idx], dtype=torch.long, device=device)
        }
        return sample

keypoint_dataset = ProcessData(df)

train_size = int(0.8 * len(keypoint_dataset))
val_size = len(keypoint_dataset) - train_size

train_dataset, val_dataset = random_split(keypoint_dataset, [train_size, val_size])

batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print("Training dataset size:", len(train_dataset))
print("Validation dataset size:", len(val_dataset))

Training dataset size: 6399
Validation dataset size: 1600


In [7]:
import model as m
model = m.SimpleNN4().to(device)
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

dataloader = DataLoader(keypoint_dataset, batch_size = 128, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

Model's state_dict:
fc1.weight 	 torch.Size([80, 126])
fc1.bias 	 torch.Size([80])
fc2.weight 	 torch.Size([64, 80])
fc2.bias 	 torch.Size([64])
fc3.weight 	 torch.Size([32, 64])
fc3.bias 	 torch.Size([32])
fc4.weight 	 torch.Size([20, 32])
fc4.bias 	 torch.Size([20])
fc5.weight 	 torch.Size([10, 20])
fc5.bias 	 torch.Size([10])
fc6.weight 	 torch.Size([8, 10])
fc6.bias 	 torch.Size([8])


In [8]:
num_epochs = 200
previous_loss = float('inf')  # Initialize with a high value
writer = SummaryWriter()
val_loss = 0.0
for epoch in range(num_epochs+1):
    # Training loop
    model.train()
    total_correct = 0
    total_samples = 0
    if epoch != 0 and epoch % 10 == 0:
        # Validation loop
        model.eval()
        val_loss = 0.0
        val_total_correct = 0
        val_total_samples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['data']
                labels = batch['label']

                outputs = model(inputs)
                val_loss += criterion(outputs, labels)

                _, predicted = torch.max(outputs, 1)
                val_total_samples += labels.size(0)
                val_total_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * val_total_correct / val_total_samples
        print(f'Validation Loss: {val_loss.item():.4f}, Training Accuracy: {train_accuracy:.2f}%, Validation Accuracy: {val_accuracy:.2f}%')
        writer.add_scalar('Validation Loss', val_loss.item(), epoch)
        writer.add_scalar('Validation Accuracy', val_accuracy, epoch)
    for batch in train_loader:
        inputs = batch['data']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    train_accuracy = 100 * total_correct / total_samples
    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}')

    writer.add_scalar('Training Loss', loss.item(), epoch)
    writer.add_scalar('Training Accuracy', train_accuracy, epoch)
    if val_loss < previous_loss:
        previous_loss = val_loss  # Update the previous loss

# Save the trained model
torch.save(model.state_dict(), "models/7_9_3.pth")

Epoch [1/200], Training Loss: 1.7521
Epoch [2/200], Training Loss: 1.5916
Epoch [3/200], Training Loss: 1.5315
Epoch [4/200], Training Loss: 1.5460
Epoch [5/200], Training Loss: 1.5306
Epoch [6/200], Training Loss: 1.5393
Epoch [7/200], Training Loss: 1.5520
Epoch [8/200], Training Loss: 1.3874
Epoch [9/200], Training Loss: 1.3980
Epoch [10/200], Training Loss: 1.4815
Validation Loss: 1.3012, Training Accuracy: 39.37%, Validation Accuracy: 48.25%
Epoch [11/200], Training Loss: 1.0896
Epoch [12/200], Training Loss: 1.4981
Epoch [13/200], Training Loss: 1.4406
Epoch [14/200], Training Loss: 1.4022
Epoch [15/200], Training Loss: 1.3673
Epoch [16/200], Training Loss: 1.3749
Epoch [17/200], Training Loss: 1.4044
Epoch [18/200], Training Loss: 1.3796
Epoch [19/200], Training Loss: 1.4871
Epoch [20/200], Training Loss: 1.4064
Validation Loss: 1.1593, Training Accuracy: 41.04%, Validation Accuracy: 57.69%
Epoch [21/200], Training Loss: 0.7323
Epoch [22/200], Training Loss: 1.4802
Epoch [23/200

Run this in terminal to open tensorboard: tensorboard --logdir=runs
